In [3]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv("./Data/emotion.tsv", sep="\t")

print(df.columns)

null_idx = df[df["document"].isnull()].index
df.loc[null_idx]

train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

print("중복 제거 전 학습 데이터셋: {}".format(len(train_data)))
print("중복 제거 전 테스트 데이터셋: {}".format(len(test_data)))

train_data = train_data.drop_duplicates(["document"])
test_data = test_data.drop_duplicates(["document"])

print("중복 제거 후 학습 데이터셋: {}".format(len(train_data)))
print("중복 제거 후 테스트 데이터셋: {}".format(len(test_data)))

MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_train_sentences = tokenizer(
    list(train_data['document']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
)

tokenizer_test_sentences = tokenizer(
    list(test_data['document']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
)


class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
train_label = train_data['label'].values
test_label = test_data['label'].values

train_dataset = CurseDataset(tokenizer_train_sentences, train_label)
test_dataset = CurseDataset(tokenizer_test_sentences, test_label)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(device)

traning_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="wegihted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
}

trainer = Trainer(
    model=model,
    args=traning_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Index(['document', 'label'], dtype='object')
중복 제거 전 학습 데이터셋: 240525
중복 제거 전 테스트 데이터셋: 60131
중복 제거 후 학습 데이터셋: 234389
중복 제거 후 테스트 데이터셋: 59363


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
trainer.train()

# trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/36625 [00:00<?, ?it/s]

C:\Users\jhy97\AppData\Local\Temp\ipykernel_25256\1735328254.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.6509, 'grad_norm': 7.891141891479492, 'learning_rate': 5e-05, 'epoch': 0.07}


C:\Users\jhy97\AppData\Local\Temp\ipykernel_25256\1735328254.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.3521, 'grad_norm': 5.184708595275879, 'learning_rate': 4.930795847750865e-05, 'epoch': 0.14}


...

C:\Users\jhy97\AppData\Local\Temp\ipykernel_25256\1735328254.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.037, 'grad_norm': 3.1320888996124268, 'learning_rate': 1.730103806228374e-07, 'epoch': 4.98}


C:\Users\jhy97\AppData\Local\Temp\ipykernel_25256\1735328254.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'train_runtime': 15891.714, 'train_samples_per_second': 73.746, 'train_steps_per_second': 2.305, 'train_loss': 0.13708669007597524, 'epoch': 5.0}


TrainOutput(global_step=36625, training_loss=0.13708669007597524, metrics={'train_runtime': 15891.714, 'train_samples_per_second': 73.746, 'train_steps_per_second': 2.305, 'total_flos': 7.708861358481024e+16, 'train_loss': 0.13708669007597524, 'epoch': 5.0})

In [5]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\tokenizer.json')

In [6]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# gpu 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:

# 저장된 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        [sent],
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
        )
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1).numpy()[0]
    return result

# 데이터 로드 및 샘플링
data = pd.read_csv("Data/emotion.tsv", sep='\t')
sampled_data = data.sample(n=1000)

# 정확도 계산을 위한 변수 초기화
correct_predictions = 0

for index, row in sampled_data.iterrows():
    sentence = row['document']
    true_label = row['label']
    predicted_label = sentence_predict(sentence)
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    if true_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / len(sampled_data)
print(f"정확도: {accuracy * 100:.2f}%")

True Label: 2, Predicted Label: 2
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 1, Predicted Label: 1
True Label: 1, Predicted Label: 1
True Label: 2, Predicted Label: 2
True Label: 2, Predicted Label: 2
True Label: 2, Predicted Label: 2
True Label: 1, Predicted Label: 1
True Label: 1, Predicted Label: 1
True Label: 2, Predicted Label: 2
True Label: 2, Predicted Label: 2
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 2, Predicted Label: 2
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 2, Predicted Label: 2
True Label: 1,

In [8]:

# 저장된 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        [sent],
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}
    with torch.no_grad():
        outputs = model(**tokenized_sent)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    return predicted_label, probabilities.tolist()[0]

# 데이터 로드 및 샘플링
data = pd.read_csv("Data/emotion.tsv", sep='\t')
sampled_data = data.sample(n=1000)

# 정확도 계산을 위한 변수 초기화
correct_predictions = 0

for index, row in sampled_data.iterrows():
    sentence = row['document']
    true_label = row['label']
    predicted_label, probabilities = sentence_predict(sentence)
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}, Probabilities: {probabilities}")
    if true_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / len(sampled_data)
print(f"정확도: {accuracy * 100:.2f}%")

True Label: 0, Predicted Label: 0, Probabilities: [0.9995220899581909, 0.00046339459368027747, 1.459473332943162e-05]
True Label: 2, Predicted Label: 2, Probabilities: [0.0007370602688752115, 0.00032469132565893233, 0.998938262462616]
True Label: 1, Predicted Label: 1, Probabilities: [0.0005135929677635431, 0.9994556307792664, 3.073038169532083e-05]
True Label: 1, Predicted Label: 1, Probabilities: [0.0013506972463801503, 0.9985790252685547, 7.03043770045042e-05]
True Label: 0, Predicted Label: 0, Probabilities: [0.999535083770752, 0.00044909186544828117, 1.589075691299513e-05]
True Label: 1, Predicted Label: 1, Probabilities: [0.00042689903057180345, 0.9995551705360413, 1.7881375242723152e-05]
True Label: 2, Predicted Label: 2, Probabilities: [5.797890844405629e-05, 2.134776696038898e-05, 0.9999207258224487]
True Label: 1, Predicted Label: 1, Probabilities: [0.0004726641927845776, 0.999512791633606, 1.4578280570276547e-05]
True Label: 1, Predicted Label: 1, Probabilities: [0.000438148

In [9]:

# 저장된 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        [sent],
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}
    with torch.no_grad():
        outputs = model(**tokenized_sent)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    return predicted_label, probabilities.tolist()[0]

corrected_texts = [
    "초등학교는 완전히 끝난거 맞죠?",
    "초등학교 해당되는 강사분들 답변바랍니다",
    "제가알기론 13일날 한번 남은걸로 알고있습니다.!",
    "추가로 스마트팜 조립은 프레임 빼고 다 해놨고 오늘 학생 안와서 5개만 되어있어요.",
    "각 조립에 학생들 이름 붙여놨으니 구별할 수 있습니다.",
    "그냥 그대로 하루에 2차시씩 한거 맞나요?",
    "결과보고서 및 기증문서 대부분은 제가 다 작성해놓았으니 서명받고 사업단에 제출하시면 됩니다",
    "네. 원래 일정상 13일이 마지막 날입니다",
    "안녕하세요, 11월 말일까지 진행했던 해봄학교 교육활동 출석부만 먼저 제출 부탁드립니다~ ",
    "현재 다 완료되지 않은 관계로 활동결과보고서 전체가 아닌 서명부 부탁드립니다. ",
    "지금 현재 본인이 담당하고 있는 클래스 주 강사는 현재까지 '출석부' 날짜 잘 보이게 사진 찍어서 내일 10:00시까지 보내주세요.",
    "출석부 12월 1일까지의 출결사항 및 근무일자 기재 후 필히 보내주시기 바랍니다.",
    "학교 담당자입니다.",
    "내일 회식 참석 전",
    "또는 귀가 전",
    "시발",
    "난 너가 너무 싫어",
    "혐오스러워",
    "진짜 세상 다 안망하네~ 인생 좆같네 진짜",
    "싫다 진자 전부 다 귀찮다 이젠",
    "너가 너무 미워서 정말 시러",
    "죽어",
    "살아",
    "네가 너무 좋아",
    "네가 너무 싫어",
    "행복한 감정",
    "불행한 감정",
    "하... 슬프다 진짜",
]

for text in corrected_texts:
    probabilities = sentence_predict(text)
    print(f"Text: {text}, Probabilities: {probabilities}")

Text: 초등학교는 완전히 끝난거 맞죠?, Probabilities: (2, [0.0009647451806813478, 0.0003934853884857148, 0.9986417889595032])
Text: 초등학교 해당되는 강사분들 답변바랍니다, Probabilities: (2, [0.0006567123928107321, 0.00032969730091281235, 0.9990136623382568])
Text: 제가알기론 13일날 한번 남은걸로 알고있습니다.!, Probabilities: (1, [0.0005109758931212127, 0.9994677901268005, 2.126005529135e-05])
Text: 추가로 스마트팜 조립은 프레임 빼고 다 해놨고 오늘 학생 안와서 5개만 되어있어요., Probabilities: (2, [0.00010992071474902332, 3.600112540880218e-05, 0.9998540878295898])
Text: 각 조립에 학생들 이름 붙여놨으니 구별할 수 있습니다., Probabilities: (2, [9.724044502945617e-05, 3.0096547561697662e-05, 0.9998725652694702])
Text: 그냥 그대로 하루에 2차시씩 한거 맞나요?, Probabilities: (2, [0.0001431971468264237, 4.2154966649832204e-05, 0.9998146891593933])
Text: 결과보고서 및 기증문서 대부분은 제가 다 작성해놓았으니 서명받고 사업단에 제출하시면 됩니다, Probabilities: (2, [0.00017101774574257433, 5.873238478670828e-05, 0.9997702240943909])
Text: 네. 원래 일정상 13일이 마지막 날입니다, Probabilities: (2, [0.00015919224824756384, 5.482942651724443e-05, 0.9997859597206116])
